# Data Preparation

In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-metadata.tgz > data_annotation.tgz # real annotation

In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-samples.tgz > data_image.tgz # real images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.2G  100 15.2G    0     0  14.4M      0  0:17:57  0:17:57 --:--:-- 12.4M


In [1]:
from google.colab import drive # the real images data files take too long (about 17 minutes) to be downloaded, so we just mount it
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!tar -xzf /content/drive/MyDrive/nuimages/nuimages-v1.0-all-metadata.tgz && tar -xzf /content/drive/MyDrive/nuimages/nuimages-v1.0-all-samples.tgz # extract. this will take about 7 minutes

In [3]:
drive.flush_and_unmount()

In [ ]:
!rm -rf train/ test/ val/ train.txt test.txt val.txt # remove old files

In [4]:
!mkdir train test val && mkdir train/images test/images val/images train/labels test/labels val/labels # for the yolo format

In [5]:
import sys
sys.path.append('/content/') # path where data_preparation.py is

# import importlib # use this if you make changes to the data_preparation.py and want to reflect the changes in the notebook
# importlib.reload(data_preparation)

import data_preparation

In [6]:
path_prefix = '/content/v1.0-val/'
val_df, _ = data_preparation.data_json_to_joined_df_and_class_list('/content/v1.0-val/', 1) # only take 1% just for testing
train_df, class_list = data_preparation.data_json_to_joined_df_and_class_list('/content/v1.0-train/', 1)

In [10]:
class_list

array(['animal', 'human.pedestrian.adult', 'human.pedestrian.child',
       'human.pedestrian.construction_worker',
       'human.pedestrian.personal_mobility',
       'human.pedestrian.police_officer', 'human.pedestrian.stroller',
       'human.pedestrian.wheelchair', 'movable_object.barrier',
       'movable_object.debris', 'movable_object.pushable_pullable',
       'movable_object.trafficcone', 'static_object.bicycle_rack',
       'vehicle.bicycle', 'vehicle.bus.bendy', 'vehicle.bus.rigid',
       'vehicle.car', 'vehicle.construction', 'vehicle.ego',
       'vehicle.emergency.ambulance', 'vehicle.emergency.police',
       'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck'],
      dtype=object)

In [ ]:
!rm train.txt val.txt

In [8]:
# NOTE: do NOT run df_to_yolo_format_txt to mounted Drive directory, it's slow. Avoid read/writes to mounted Drive directory
data_preparation.create_image_filename_list_txt('train', train_df)
data_preparation.create_image_filename_list_txt('val', val_df)

The text file (train/val.txt) already exists and is already filled. Remove the txt file first to make sure we start clean.
The text file (train/val.txt) already exists and is already filled. Remove the txt file first to make sure we start clean.


In [13]:
data_preparation.df_to_yolo_format_txt('/content/', 'train', class_list,train_df)
data_preparation.df_to_yolo_format_txt('/content/', 'val', class_list, val_df)

In [14]:
data_preparation.create_data_yml('/content/',class_list)

In [15]:
# make sure the val.txt exists before running this
# copy the val image to directory expected by yolo. should take about 1 minute for the entire dataset
!xargs -a val.txt -P8 -I{} cp "/content/{}" /content/val/images/

In [16]:
# make sure the train.txt exists before running this
# copy the train image to directory expected by yolo. should take about 5 minutes for the entire dataset
!xargs -a train.txt -P8 -I{} cp "/content/{}" /content/train/images/

In [ ]:
train_df['filename'].head()

,filename
0,samples/CAM_BACK_RIGHT/n009-2018-05-08-15-52-4...
1,samples/CAM_BACK/n016-2018-09-10-11-01-25+0800...
2,samples/CAM_BACK_LEFT/n016-2018-07-02-11-25-51...
3,samples/CAM_FRONT_RIGHT/n015-2018-09-05-11-47-...
4,samples/CAM_FRONT_LEFT/n008-2018-06-04-16-30-0...


In [ ]:
!ls -1 /content/val/images/ | wc -l

148


In [ ]:
!ls -1 /content/val/labels/ | wc -l

148


In [17]:
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [18]:
!yolo detect train model=yolov8n.pt data=data.yml epochs=50 imgsz=640 batch=16 project=coba

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 122MB/s]
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke

In [33]:
from ultralytics import YOLO

import os
import random

def get_random_file(directory):
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None  # or raise an error if preferred
    return os.path.join(directory, random.choice(files))

# Example usage
random_file = get_random_file("/content/samples/CAM_FRONT_RIGHT/")
# 1) Load the fine-tuned weights
model = YOLO("/content/coba/train/weights/best.pt")

model.predict(
    source=random_file,  # or a local path / folder
    conf=0.25,
    save=True,        # annotated image saved to runs/detect/predict/
    imgsz=640
)
print("Prediction saved to runs/detect/predict/")



image 1/1 /content/samples/CAM_FRONT_RIGHT/n010-2018-07-23-11-50-54+0800__CAM_FRONT_RIGHT__1532318074170905.jpg: 384x640 9 human.pedestrian.adults, 1 static_object.bicycle_rack, 2 vehicle.bicycles, 1 vehicle.car, 7.6ms
Speed: 2.2ms preprocess, 7.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15
Prediction saved to runs/detect/predict/
